# Лабораторная работа 3

---

Загрузка необходимых библиотек:

In [1]:
import requests
import zipfile
import io
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.dates as mdates

Глобальные перменные, функции и настройки:

In [11]:
raw_data_path = "data/raw/household_power_consumption.csv2"
clean_data_path = "data/clean/Бытовое потребление электрической энергии.csv2"
pd.options.display.max_rows = 8 # Отображение максимум 8 строк в Pandas dataframe
mkdate_dec = lambda x: mdates.date2num(dt.datetime.strptime(x.decode("utf-8"), "%d/%m/%Y")) # Дата в число с декодированием
mktime_dec = lambda x: mdates.date2num(dt.datetime.strptime(x.decode("utf-8"), "%H:%M:%S")) # Время в число с декодированием
mktime = lambda x: mdates.date2num(dt.datetime.strptime(x, "%H:%M:%S")) # Время в число
pd_mktime = lambda x: mktime(x["Время"])

Загрузка и очистка данных:

In [3]:
request = requests.get("https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip") # Загрузка файла
file = zipfile.ZipFile(io.BytesIO(request.content)) # Разархивирование
with open(raw_data_path, "w") as f_raw: # Открыть файл для записи "грязных" данных
    with open(clean_data_path, "w") as f_clean: # Открыть файл для записи "чистых" данных
        for line in file.open("household_power_consumption.txt").readlines(): # Считать построчно файл из архива
            line_en = line.decode("utf-8") # Декодирование строчки, формат - UTF-8
            f_raw.write(line_en) # Запись строчки в файл с "грязными" данными
            if "?" not in line_en: # Проверка на наличие некорректных данных
                if line_en[0] == "D": # Если это первая строчка, то заменить ее
                    f_clean.write("Дата;Время;Активная_мощьность;Реактивная_мощность;Напряжение;Средняя_сила_тока;Набор_потребителей_1;Набор_потребителей_2;Набор_потребителей_3\n")
                else:
                    f_clean.write(line_en) # В случае корректных данных, записать строчку в файл с "чистыми" данными
        file.close() # Закрыть файл архива

Занесение данных в *Pandas dataframe*:

In [9]:
pd_data = pd.read_csv(clean_data_path, sep=";", header=0, low_memory=False)
pd_data["Время"] = pd_data.apply(pd_mktime, axis=1)

Занесение данных в *NumPy array*:

In [12]:
np_data = np.loadtxt(clean_data_path, skiprows=1, 
                     dtype={"names": ("Дата","Время","Активная_мощьность","Реактивная_мощность","Напряжение","Средняя_сила_тока","Набор_потребителей_1","Набор_потребителей_2","Набор_потребителей_3"), "formats": ("f8", "f8", "f8", "f8", "f8", "f8", "f8", "f8", "f8")}, 
                     delimiter=";", 
                     converters={0:mkdate_dec, 1:mktime_dec}, # 0:mkdate_dec, 
                     ndmin=0)

**1**) Подсчет времени выполнения выбора домохазяйств с активной потребительной мощьностью выше 5 кВт:

In [13]:
pd_task1 = %timeit -c -q -o pd_data.loc[(pd_data["Активная_мощьность"] > 5)] # Pandas dataframe
pd_task1_time = pd_task1.best
np_task1 = %timeit -c -q -o np_data[np_data["Активная_мощьность"] > 5] # NumPy array
np_task1_time = np_task1.best

**2**) Подсчет времени выполнения выбора домохазяйств, у которых вольтаж больше 235 В:

In [14]:
pd_task2 = %timeit -c -q -o pd_data.loc[(pd_data["Напряжение"] > 235)] # Pandas dataframe
pd_task2_time = pd_task2.best
np_task2 = %timeit -c -q -o np_data[np_data["Напряжение"] > 235] # NumPy array
np_task2_time = np_task2.best

**3**) Подсчет времени фильтрации всех домохозяйства у которых сила тока 19-20 А, а стиральная машина и холодильник потребляет больше, чем бойлер и кондиционер:

In [15]:
pd_task3 = %timeit -c -q -o pd_data.loc[(pd_data["Средняя_сила_тока"] >= 19) & (pd_data["Средняя_сила_тока"] <= 20) & (pd_data["Набор_потребителей_2"] > pd_data["Набор_потребителей_3"])] # Pandas dataframe
pd_task3_time = pd_task3.best
np_task3 = %timeit -c -q -o np_data[(np_data["Средняя_сила_тока"] >= 19)& (np_data["Средняя_сила_тока"] <= 20) & (np_data["Набор_потребителей_2"] > np_data["Набор_потребителей_3"])] # NumPy array
np_task3_time = np_task3.best

**4**) Подсчет времени выбора *500 000* домохазяйств случайным образом без повторения, для них вычислить средние значения 3 групп потребителей электрической энергии:

In [16]:
pd_task4 = %timeit -c -q -o temp = pd_data.sample(n=500000); temp["Набор_потребителей_1"].mean(); temp["Набор_потребителей_2"].mean(); temp["Набор_потребителей_3"].mean()
pd_task4_time = pd_task4.best
np_task4 = %timeit -c -q -o temp = np.random.choice(np_data, size=500000); np.mean(temp["Набор_потребителей_1"]); np.mean(temp["Набор_потребителей_2"]); np.mean(temp["Набор_потребителей_3"])
np_task4_time = np_task4.best

**5**) Засекам: из выбранных случайным образом *500 000* домохазяйств выбрать те, которые после *18-00* потребляют в среднем более 6 кВт в минуту. Среди них выбрать те, у которых 2 группа потребления является больше остальных. Среди них выбрать каждый третий результат из первой половины и каждый четвертый из второй половины:

In [66]:
time18 = mktime("18:00:00")
pd_rand = pd_data.sample(n=500000)
np_rand = np.random.choice(np_data, size=500000)

pd_task5 = %timeit -c -q -o temp = pd_rand.loc[(pd_rand["Время"] >= time18) & (pd_rand["Напряжение"] > 6) & (pd_rand["Набор_потребителей_2"] > pd_rand["Набор_потребителей_1"]) & (pd_rand["Набор_потребителей_2"] > pd_rand["Набор_потребителей_3"])]; temp.head(round(len(temp.index)/2)).iloc[::3, :]; temp.tail(round(len(temp.index)/2)).iloc[::4, :]
pd_task5_time = pd_task5.best
np_task5 = %timeit -c -q -o temp = np_rand[(np_rand["Время"] >= time18) & (np_rand["Напряжение"] > 6) & (np_rand["Набор_потребителей_2"] > np_rand["Набор_потребителей_1"]) & (np_rand["Набор_потребителей_2"] > np_rand["Набор_потребителей_3"])]; temp[0:round(temp.shape[0]/2):3]; temp[round(temp.shape[0]/2):temp.shape[0]:4]
np_task5_time = np_task5.best

Результаты оформим в виде таблицы:

In [97]:
results = [
    (pd_task1_time, np_task1_time),
    (pd_task2_time, np_task2_time),
    (pd_task3_time, np_task3_time),
    (pd_task4_time, np_task4_time),
    (pd_task5_time, np_task5_time)
]
df = pd.DataFrame(results, columns=["Pandas dataframe, c", "NumPy array, c"])
df.index = ("Задание 1", "Задание 2", "Задание 3", "Задание 4", "Задание 5")
df["Кто быстрее?"] = df["Pandas dataframe, c"] > df["NumPy array, c"]
df["Кто быстрее?"] = df["Кто быстрее?"].map({False:"NumPy array", True:"Pandas dataframe"})
df["Разница, с"] = abs(df["Pandas dataframe, c"] - df["NumPy array, c"])
df

,"Pandas dataframe, c","NumPy array, c",Кто быстрее?,"Разница, с"
Задание 1,0.007164,0.026840,NumPy array,0.019675
Задание 2,0.139842,0.089750,Pandas dataframe,0.050092
Задание 3,0.023070,0.067970,NumPy array,0.044900
Задание 4,0.283425,0.279195,Pandas dataframe,0.004230
Задание 5,0.016111,0.023830,NumPy array,0.007720


### Вывод

Во время выполнения данной лабараторной работы я угулубил знания по работе с *Pandas dataframes* и узнал принципы работы с *NumPy arrays*.
Кроме того, исследовал скорость работы обоих этих инструментов. По результатом этой проверки получилось что *NumPy* в большем колличестве заданий оказался быстрее *Pandas*. Безусловно, для обработки *Big Data*, следует использовать *NumPy*, но для обработки небольшой информации *Pandas*, по моему мнению, подходит намного лучше, с точки зрения удобства использования. Моя итоговая оценка: NumPy - 4; Pandas - 5.